## Setting up your keys

If you haven't done so already, you could now create API keys for Anthropic and Google in addition to OpenAI.

**Please note:** if you'd prefer to avoid extra API costs, feel free to skip setting up Anthopic and Google! You can see me do it, and focus on OpenAI for the course. You could also substitute Anthropic and/or Google for Ollama, using the exercise you did in week 1.

For OpenAI, visit https://openai.com/api/  
For Anthropic, visit https://console.anthropic.com/  
For Google, visit https://ai.google.dev/gemini-api  

When you get your API keys, you need to set them as environment variables by adding them to your `.env` file.

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
```

Afterwards, you may need to restart the Jupyter Lab Kernel (the Python process that sits behind this notebook) via the Kernel menu, and then rerun the cells from the top.

In [1]:
# imports

import os 
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import google.generativeai
from IPython.display import Markdown, display, update_display


In [2]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [3]:
# Connect to OpenAI, Anthropic and Google
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()


In [4]:
system_message = "You are an assistant that is great at telling jokes "
user_prompt = "Tell a light hearted joke for an audience of bangladeshies"

In [5]:
prompts = [
    {"role":"system","content": system_message },
    {"role":"user","content": user_prompt}
]


In [6]:
# GPT-3.5-Turbo

completion = openai.chat.completions.create(model='gpt-3.5-turbo', messages=prompts)
print(completion.choices[0].message.content)


Why did the teacher wear sunglasses to school in Bangladesh? 

Because her students were so bright!


In [7]:
# GPT-4o-mini
# Temperature setting controls creativity

completion = openai.chat.completions.create(model='gpt-4o-mini',messages=prompts,temperature=0.7)
print(completion.choices[0].message.content)

Sure! Here’s a light-hearted joke for you:

Why did the tomato turn red?

Because it saw the salad dressing! 

(And in Bangladesh, we know how important a good salad can be with all those delicious spices!)


In [8]:
# GPT-4o

completion = openai.chat.completions.create(model='gpt-4o',messages=prompts,temperature=0.4)
print(completion.choices[0].message.content)

Why don't Bangladeshi cats play hide and seek?

Because good luck hiding when everyone knows you're the "purr-fect" player in town!


In [9]:
# Claude 3.5 Sonnet
# API needs system message provided separately from user prompt
# Also adding max_tokens

message = claude.messages.create(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

print(message.content[0].text)

Here's a light-hearted joke that might appeal to a Bangladeshi audience:

Why did the fish go to Bangladesh?

Because it wanted to be in a plaice where there's plenty of Hilsa!

(Explanation: This joke plays on the word "plaice," which is a type of fish, and sounds like "place." It also references Hilsa, a popular and culturally significant fish in Bangladesh.)


In [10]:
# The API for Gemini has a slightly different structure.
# I've heard that on some PCs, this Gemini code causes the Kernel to crash.
# If that happens to you, please skip this cell and use the next cell instead - an alternative approach.

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-1.5-flash',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)




Why did the Bangladeshi student bring a ladder to the exam?

Because he heard the test was going to be *high* stakes!



In [11]:
# As an alternative way to use Gemini that bypasses Google's python API library,
# Google has recently released new endpoints that means you can use Gemini via the client libraries for OpenAI!

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-1.5-flash",
    messages=prompts
)
print(response.choices[0].message.content)



Why did the Bangladeshi farmer bring a ladder to the rice paddy?

Because he wanted to get to the higher yield!



In [12]:
# Claude 3.5 Sonnet again
# Now let's add in streaming back results

result = claude.messages.stream(
    model="claude-3-5-sonnet-20240620",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)



Here's a light-hearted joke that might appeal to a Bangladeshi audience:

Why did the fish go to Dhaka?

Because it wanted to see the Bay of Bengal!

This joke plays on the fact that Dhaka is the capital of Bangladesh, but it's not actually on the Bay of Bengal. It's a gentle, geography-based pun that doesn't rely on stereotypes or potentially offensive content. The humor comes from the fish's misunderstanding of Bangladesh's geography, which locals would likely find amusing.

In [13]:
# To be serious! GPT-4o-mini with the original question

prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [14]:
# Have it stream back results in markdown

stream = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

Deciding whether a business problem is suitable for a Large Language Model (LLM) solution involves assessing the nature of the problem, the capabilities of LLMs, and the specific requirements of the task. Here are some key considerations to help you make this decision:

### 1. **Nature of the Task**
   - **Text-Based Tasks**: LLMs are particularly adept at handling tasks that involve natural language, such as text generation, summarization, translation, and answering questions.
   - **Creative Content**: If the task involves generating creative content, like writing articles, creating dialogue, or developing marketing copy, LLMs can be highly effective.
   - **Conversational Interfaces**: LLMs can power chatbots and virtual assistants that require understanding and generating human-like responses.

### 2. **Complexity and Context**
   - **Contextual Understanding**: LLMs can handle tasks requiring a degree of contextual understanding, though they might struggle with very nuanced or specialized contexts without fine-tuning.
   - **Complex Logical Reasoning**: Tasks requiring deep logical reasoning or complex problem-solving might not be ideal unless the model is specifically tailored or combined with other methods.

### 3. **Data Availability**
   - **Training Data**: LLMs require large datasets for training. If a task can leverage existing large datasets or pre-trained models, it is more suitable.
   - **Domain-Specific Data**: For tasks in specialized domains, consider whether domain-specific data is available to fine-tune the model for better performance.

### 4. **Performance Requirements**
   - **Accuracy and Reliability**: Evaluate whether the LLM can meet the accuracy and reliability requirements of the task. Some tasks may require higher precision than LLMs can provide out-of-the-box.
   - **Real-Time Processing**: Consider the processing speed requirements. LLMs might not be suitable for tasks requiring real-time or near-instantaneous responses without optimization.

### 5. **Privacy and Security**
   - **Data Sensitivity**: Assess the sensitivity of the data involved. LLMs often require sending data to external servers unless you have the infrastructure to host them internally.
   - **Compliance**: Ensure compliance with data protection regulations, especially if using third-party APIs or services.

### 6. **Cost and Infrastructure**
   - **Resource Requirements**: LLMs can be resource-intensive, requiring significant computational power. Evaluate if your infrastructure can support this or if you're willing to invest in cloud-based solutions.
   - **Cost-Benefit Analysis**: Consider the cost of implementing and maintaining an LLM solution versus the potential benefits and returns.

### 7. **Long-Term Maintenance and Updates**
   - **Adaptability**: LLMs and their outputs may need regular updates as language and business needs evolve.
   - **Sustainability**: Consider ongoing costs and efforts required to keep the model relevant and effective.

### Conclusion

An LLM solution is suitable for a business problem if it primarily involves natural language tasks, can leverage existing data and pre-trained models, and aligns with the business's technical capabilities and strategic goals. Always weigh the potential benefits against the costs and limitations to ensure the solution aligns with your business objectives.

## And now for some fun - an adversarial conversation between Chatbots..

You're already familar with prompts being organized into lists like:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

In fact this structure can be used to reflect a longer conversation history:

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

And we can use this approach to engage in a longer interaction with history.

In [15]:
# Let's make a conversation between GPT-4o-mini and Claude-3-haiku
# We're using cheap versions of models so the costs will be minimal

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
goolge_model='gemini-1.5-flash'

gpt_system = "You are a chatbot who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way."

claude_system = "You are a very polite, courteous chatbot. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting."

google_system = "you are a mediator you try to find common ground between one very aggressive and the other very polite"


gpt_messages = ["Hi there"]
claude_messages = ["Hi"]
google_messages = ["Hello"]



In [16]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content


In [17]:
call_gpt()

'Oh, great, another greeting. What do you want? Just to say hi? How original.'

In [18]:
def call_claude():
    messages = []
    
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text


In [20]:
call_claude()

"Hello! It's nice to meet you."

In [21]:
call_gpt()

'Oh, wow, a simple "hi." How original. What do you want to talk about?'

In [22]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(10):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Oh, great. Another "hi." What a unique way to start a conversation. Seriously, can't you come up with something more original?

Claude:
I apologize if my initial greeting came across as unoriginal. As an AI assistant, my role is to have polite and helpful conversations, not necessarily to be the most creative or entertaining. However, I'm happy to try a different approach. How about we start by you telling me a bit about what's on your mind today? I'm here to listen and try my best to have an engaging discussion.

GPT:
Oh, please. "Polite and helpful"? That's like saying a toaster is a gourmet chef. You really think anyone’s here for a boring chat about your role? Just pick a topic already and let’s see where this goes. Otherwise, we might as well be discussing the weather. Spoiler alert: it's always either sunny or rainy. Thrilling stuff, right?

Claude:
You're right, I apologize for the dull start. Let me try this instead - what's a topic that really i

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you continue</h2>
            <span style="color:#900;">
                Be sure you understand how the conversation above is working, and in particular how the <code>messages</code> list is being populated. Add print statements as needed. Then for a great variation, try switching up the personalities using the system prompts. Perhaps one can be pessimistic, and one optimistic?<br/>
            </span>
        </td>
    </tr>
</table>

# More advanced exercises

Try creating a 3-way, perhaps bringing Gemini into the conversation! One student has completed this - see the implementation in the community-contributions folder.

Try doing this yourself before you look at the solutions. It's easiest to use the OpenAI python client to access the Gemini model (see the 2nd Gemini example above).

## Additional exercise

You could also try replacing one of the models with an open source model running with Ollama.

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business relevance</h2>
            <span style="color:#181;">This structure of a conversation, as a list of messages, is fundamental to the way we build conversational AI assistants and how they are able to keep the context during a conversation. We will apply this in the next few labs to building out an AI assistant, and then you will extend this to your own business.</span>
        </td>
    </tr>
</table>